In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("")

chat = ChatOpenAI(
    temperature=0.1
)

messages = [
    SystemMessage(content="Your are a geography expert, Ant you only reply in {language}"),
    AIMessage(content="Ciao, mi chiamo {name}"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi in questa lingua. Il mio nome è Ant. Come posso aiutarti con la geografia?')

In [2]:
#Prompt Templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)
# template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

# prompt = template.format(country_a = "Mexico", country_b = "Thailand")
# chat.predict(prompt)

template = ChatPromptTemplate.from_messages([
    ("system", "Your are a geography expert, Ant you only reply in {language}"),
    ("ai", "Ciao, mi chiamo {name}"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a ="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [2]:
# OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items= text.strip().split(",")
        return list(map(str.strip, items))


template = ChatPromptTemplate.from_messages([
    ("system", "Your are a list generathing machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 10,
    "question":"What are the planets",
})




['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [3]:
#Chaining Chains

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."), 
    ("human", "{recipe}")
])

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chef_chain

final_chain.invoke({
    'cuisine' : 'indian'
})

AIMessage(content="To make a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground turmeric\n- 1 teaspoon garam masala\n- 1 teaspoon ground coriander\n- 1 teaspoon chili powder (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or dairy-free heavy cream alternative)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use tofu or paneer instead of chicken. Marinate for at least 1 hour, or overn